# Steps to conduct

1. Setup (Databricks/AWS/GitHub)     ---------------------------                       <-- We are currently here!
2. EDA (within Databricks per PySpark/SQL)
3. Feature Selection (Databricks)
4. Write to S3
5. ML case on AWS Sagemaker (model, scores, etc.)
6. Deployment on AWS Sagemaker
6. Create presentation
7. (20:80 or optional task)

# Import Data

In [0]:
df_names = spark.read.load("dbfs:/mnt/data/TSV/name.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_akas = spark.read.load("dbfs:/mnt/data/TSV/title.akas.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_basics = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_principals = spark.read.load("dbfs:/mnt/data/TSV/title.principals.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_ratings = spark.read.load("dbfs:/mnt/data/TSV/title.ratings.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")

# EDA

## Possible list of questions (non-exhaustive):
1. What is the range of our ratings (inlcuding distribution)? --------------------------------------- DONE
2. How many votes does a movie have on average? (Box-Plot)
3. What timeframe does our dataset span? (e.g. oldest and newest movie)
4. Who are the most popular actors and directors?
5. What genres are represented the most?
6. Distribution of films / series / shows?
    ----> Focus on movies!
7. What genres are in the dataset?
8. Which genres have the highest rating?
9. Which actors play in the high rated films?
10. In which countries were the most high-rated films made and when?
11. Dependence on high rated film:
    - Country of origin ---> Actor?
    - Genre -> Actor
    - Year of creation -> Actor -> Genre
12. Which parameters go into the rating?

.....

## Imports and Functions

In [0]:
from pyspark.sql.functions import mean as _mean, \
                                  min as _min, \
                                  max as _max, \
                                  count as _count, \
                                  stddev as _stddev, col

import matplotlib.pyplot as plt
from pyspark.sql import functions as F
spark.version

Out[138]: '3.0.1'

# EDA - Basics File

In [0]:
# Import of the dataset

df_basics = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")




In [0]:
# Creating  a sample file with 50% of the dataset
df_basics_sample = df_basics.sample(False, 0.5, 42)


In [0]:
df_basics_sample.show(vertical=True)

-RECORD 0------------------------------
 tconst | tt0000004 
 titleType | short 
 primaryTitle | Un bon bock 
 originalTitle | Un bon bock 
 isAdult | 0 
 startYear | 1892 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Animation,Short 
-RECORD 1------------------------------
 tconst | tt0000008 
 titleType | short 
 primaryTitle | Edison Kinetoscop... 
 originalTitle | Edison Kinetoscop... 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 2------------------------------
 tconst | tt0000011 
 titleType | short 
 primaryTitle | Akrobatisches Pot... 
 originalTitle | Akrobatisches Pot... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 3------------------------------
 tconst | tt0000013 
 titleType | short 
 primaryTitle | The Photographica... 
 originalTitle | Neuville-sur-Saôn... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 4------------------------------
 tconst | tt0000017 
 titleType | short 
 primaryTitle | Italienischer Bau... 
 originalTitle | Italienischer Bau... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 5------------------------------
 tconst | tt0000019 
 titleType | short 
 primaryTitle | The Clown Barber 
 originalTitle | The Clown Barber 
 isAdult | 0 
 startYear | 1898 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Comedy,Short 
-RECORD 6------------------------------
 tconst | tt0000022 
 titleType | short 
 primaryTitle | Blacksmith Scene 
 originalTitle | Les forgerons 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 7------------------------------
 tconst | tt0000027 
 titleType | short 
 primaryTitle | Cordeliers' Squar... 
 originalTitle | Place des Cordeli... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 8------------------------------
 tconst | tt0000028 
 titleType | short 
 primaryTitle | Fishing for Goldfish 
 originalTitle | La pêche aux pois... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 9------------------------------
 tconst | tt0000029 
 titleType | short 
 primaryTitle | Baby's Dinner 
 originalTitle | Repas de bébé 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 10-----------------------------
 tconst | tt0000033 
 titleType | short 
 primaryTitle | Trick Riding 
 originalTitle | La voltige 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Comedy,Documentar... 
-RECORD 11-----------------------------
 tconst | tt0000038 
 titleType | short 
 primaryTitle | The Ball Game 
 originalTitle | The Ball Game 
 isAdult | 0 
 startYear | 1898 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Documentary,Short... 
-RECORD 12-----------------------------
 tconst | tt0000039 
 titleType | short 
 primaryTitle | Barnet Horse Fair 
 originalTitle | Barnet Horse Fair 
 isAdult | 0 
 startYear | 1896 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Short 
-RECORD 13-----------------------------
 tconst | tt0000040 
 titleType | short 
 primaryTitle | Barque sortant du... 
 originalTitle | Barque sortant du... 
 isAdult | 0 
 startYear | 1896 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Documentary,Short 
-RECORD 14-----------------------------
 tconst | tt0000042 
 titleType | short 
 primaryTitle | Bateau-mouche sur... 
 originalTitle | Bateau-mouche sur... 
 isAdult | 0 
 startYear | 1896 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Documentary,Short 
-RECORD 15-----------------------------
 tconst | tt0000043 
 titleType | short 
 primaryTitle | Batteuse à vapeur 
 originalTitle | Batteuse à vapeur 
 isAdult | 0 
 startYear | 1896 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Documentary,Short 
-RECO

In [0]:
# What kind of data are in the columns?

df_basics_sample.printSchema()

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

In [0]:
# Learn the data

display(df_basics_sample)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000011,short,Akrobatisches Potpourri,Akrobatisches Potpourri,0,1895,\N,1,"Documentary,Short"
tt0000013,short,The Photographical Congress Arrives in Lyon,Neuville-sur-Saône: Débarquement du congrès des photographes à Lyon,0,1895,\N,1,"Documentary,Short"
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short"
tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short"
tt0000022,short,Blacksmith Scene,Les forgerons,0,1895,\N,1,"Documentary,Short"
tt0000027,short,Cordeliers' Square in Lyon,Place des Cordeliers à Lyon,0,1895,\N,1,"Documentary,Short"
tt0000028,short,Fishing for Goldfish,La pêche aux poissons rouges,0,1895,\N,1,"Documentary,Short"
tt0000029,short,Baby's Dinner,Repas de bébé,0,1895,\N,1,"Documentary,Short"


In [0]:
# Some statistics

display(df_basics_sample.describe())

summary,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,3164566,3164566,3164566,3164566,3164566,3164566,3164566,3164566,3164562
mean,null,null,NaN,NaN,0.032295739763367234,2001.477381246509,2001.819784774231,45.02869634084998,null
stddev,null,null,NaN,NaN,2.263922474785311,21.192394317819993,16.78727073039977,78.47768412571381,null
min,tt0000004,movie,!Women Art Revolution,"""1 jikan buchi nuki de Sasuke ga ô abare dattebayo supesharu"": Date ni okureta wake janai! Kyûkyoku ôgi - Chidori tanjô!!",0,1874,1927,0,Action
max,tt9916852,videoGame,Šiška Deluxe,überRICH,2019,\N,\N,\N,\N


- Start all tconst with tt?
- Only 'isAdult' is an integer. What is the range or which values are available? (1)
- What is the content of titleTyp? (2)
- What is the difference between titleTyp and genres?
- What means nullable=true?
- Column 'genres' contains different types of genres - seperate!

(1) Only 'isAdult' is an integer. What is the range or which values are available?

In [0]:
display(df_basics_sample.select('tconst'))

tconst
tt0000004
tt0000008
tt0000011
tt0000013
tt0000017
tt0000019
tt0000022
tt0000027
tt0000028
tt0000029


In [0]:
# Distribution of 0, 1, and the year numbers

groupBy_output = df_basics_sample.groupBy("isAdult").count()
display(groupBy_output)

isAdult,count
1,94170
0,3070392
2014,1
1994,1
2019,1
2005,1


Content of the column 'isAdult' contain the values 0 and 1,sometimes a year number ----> delete all year number entries!

In [0]:
# column 'isAdult' drop rows with year number

#df_basics_sample.where(col('isAdult') == 2019).drop()

In [0]:
# GroupBy the titleTyp and count the sum

groupBy_output = df_basics_sample.groupBy("titleType").count()
display(groupBy_output)

titleType,count
tvSeries,87289
tvMiniSeries,14341
tvMovie,60361
tvEpisode,2224154
movie,268023
tvSpecial,13121
video,123721
videoGame,12392
tvShort,5758
short,355406


---> most of the movies are in the titleType tvEpisode

In [0]:
# GroupBy the genres and count the sum

groupBy_output = df_basics_sample.groupBy("titleType", "genres","isAdult").count()
display(groupBy_output)

titleType,genres,isAdult,count
short,"Biography,Romance,Short",0,11
movie,"Fantasy,Sci-Fi",0,63
movie,"Comedy,Family,Musical",0,49
movie,"Film-Noir,Mystery,Thriller",0,3
tvSeries,Comedy,0,13540
movie,"Action,Horror,Thriller",0,123
tvSeries,"Adventure,Drama,Sci-Fi",0,13
tvSeries,"Animation,Drama,Family",0,13
movie,"Fantasy,Mystery,Romance",0,11
tvSeries,"Animation,Family,Western",0,1


In [0]:
# Split of the genres column in genre 1, 2, 3

df_basics_encode = df_basics_sample.withColumn("genre1", split(col("genres"), ",").getItem(0)).withColumn("genre2", split(col("genres"), ",").getItem(1)).withColumn("genre3", split(col("genres"), ",").getItem(2))
df_basics_encode.display()

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre1,genre2,genre3
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",Animation,Short,null
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",Documentary,Short,null
tt0000011,short,Akrobatisches Potpourri,Akrobatisches Potpourri,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000013,short,The Photographical Congress Arrives in Lyon,Neuville-sur-Saône: Débarquement du congrès des photographes à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short",Comedy,Short,null
tt0000022,short,Blacksmith Scene,Les forgerons,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000027,short,Cordeliers' Square in Lyon,Place des Cordeliers à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000028,short,Fishing for Goldfish,La pêche aux poissons rouges,0,1895,\N,1,"Documentary,Short",Documentary,Short,null
tt0000029,short,Baby's Dinner,Repas de bébé,0,1895,\N,1,"Documentary,Short",Documentary,Short,null


In [0]:
# Transform genre columns into StringIndexer

from pyspark.ml.feature import StringIndexer


#input = df_basics_encode['genre1']

indexer = StringIndexer(inputCol='genre1', outputCol='genre1index')



indexed = indexer.fit(df_basics_encode).transform(df_basics_encode)
indexed.display()

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre1,genre2,genre3,genre1index
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",Animation,Short,null,7.0
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000011,short,Akrobatisches Potpourri,Akrobatisches Potpourri,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000013,short,The Photographical Congress Arrives in Lyon,Neuville-sur-Saône: Débarquement du congrès des photographes à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short",Comedy,Short,null,1.0
tt0000022,short,Blacksmith Scene,Les forgerons,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000027,short,Cordeliers' Square in Lyon,Place des Cordeliers à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000028,short,Fishing for Goldfish,La pêche aux poissons rouges,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0
tt0000029,short,Baby's Dinner,Repas de bébé,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0


In [0]:
# OneHotEncoding of genre1, genre2 and genre3

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["genre1index"],
                        outputCols=["genral1index-onehot"])
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.display()




tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre1,genre2,genre3,genre1index,genral1index-onehot
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",Animation,Short,null,7.0,"List(0, 28, List(7), List(1.0))"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000011,short,Akrobatisches Potpourri,Akrobatisches Potpourri,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000013,short,The Photographical Congress Arrives in Lyon,Neuville-sur-Saône: Débarquement du congrès des photographes à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000019,short,The Clown Barber,The Clown Barber,0,1898,\N,\N,"Comedy,Short",Comedy,Short,null,1.0,"List(0, 28, List(1), List(1.0))"
tt0000022,short,Blacksmith Scene,Les forgerons,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000027,short,Cordeliers' Square in Lyon,Place des Cordeliers à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000028,short,Fishing for Goldfish,La pêche aux poissons rouges,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"
tt0000029,short,Baby's Dinner,Repas de bébé,0,1895,\N,1,"Documentary,Short",Documentary,Short,null,2.0,"List(0, 28, List(2), List(1.0))"


Next steps:

-delete columns
- |-- tconst: string (nullable = true) 
- |-- titleType: string (nullable = true)
- |-- primaryTitle: string (nullable = true) ---> delete column
- |-- originalTitle: string (nullable = true)---> delete column
- |-- isAdult: integer (nullable = true)---> delete
- |-- startYear: string (nullable = true)
- |-- endYear: string (nullable = true) --->delete column
- |-- runtimeMinutes: string (nullable = true)
- |-- genres: string (nullable = true)---> delete /n




-cleanig nan /N
- |-- tconst: string (nullable = true) 
- |-- titleType: string (nullable = true)
- |-- primaryTitle: string (nullable = true) ---> delete column
- |-- originalTitle: string (nullable = true)
- |-- isAdult: integer (nullable = true)---> delete column
- |-- startYear: string (nullable = true)
- |-- endYear: string (nullable = true) --->delete column
- |-- runtimeMinutes: string (nullable = true)
- |-- genres: string (nullable = true)---> delete /n


- Encoding of Genres

In [0]:
#drop nan rows
df_basics_sample.where(col('isAdult') == "2019").drop().show()

+---------+---------+--------------------+-------------+-------+---------+-------+--------------+------+
 tconst|titleType| primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|
+---------+---------+--------------------+-------------+-------+---------+-------+--------------+------+
tt9822816|tvEpisode|"Zwischen Vertusc...| 0| 2019| \N| \N| \N| null|
+---------+---------+--------------------+-------------+-------+---------+-------+--------------+------+

In [0]:
#drop nan rows
df_basics_sample.where(fr'{column} = "r\N"').drop()

Out[154]: DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: int, startYear: string, endYear: string, runtimeMinutes: string, genres: string]

In [0]:
display(
  df_basics_sample.select('isAdult').distinct()
)

isAdult
1
0
2014
1994
2019
2005


In [0]:
df_basics_sample.printSchema()

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

In [0]:
# Splitting of the columns genres and encoding 

from pyspark.sql.functions import split, array_distinct, explode

df_basics_sample.select('tconst', 'genres', explode(array_distinct(split('genres', ',')))).groupBy('tconst').pivot('col').count().fillna(0).show()

+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
 tconst|Action|Adult|Adventure|Animation|Biography|Comedy|Crime|Documentary|Drama|Family|Fantasy|Film-Noir|Game-Show|History|Horror|Music|Musical|Mystery|News|Reality-TV|Romance|Sci-Fi|Short|Sport|Talk-Show|Thriller|War|Western| \N|
+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
tt0013305| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0089167| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0235483| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0245237| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0331671| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0349113| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0493282| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0544067| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0674313| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0716421| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0028810| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0090978| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
tt0220939| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
tt0336404| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0386313| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0421120| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0440947| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0505332| 0| 0| 0| 0| 0| 1| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0566470| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0592174| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
only showing top 20 rows

In [0]:

df_basics_sample.count()

Out[158]: 3164566

In [0]:
df_basics_sample.distinct().count()

Out[159]: 3164566

In [0]:
droplist = ['primaryTitle', 'originalTitle', 'endYear']
df_basics_sample_fs = df_basics_sample.drop(*droplist)
df_basics_sample_fs.show()

+---------+---------+-------+---------+--------------+--------------------+
 tconst|titleType|isAdult|startYear|runtimeMinutes| genres|
+---------+---------+-------+---------+--------------+--------------------+
tt0000004| short| 0| 1892| \N| Animation,Short|
tt0000008| short| 0| 1894| 1| Documentary,Short|
tt0000011| short| 0| 1895| 1| Documentary,Short|
tt0000013| short| 0| 1895| 1| Documentary,Short|
tt0000017| short| 0| 1895| 1| Documentary,Short|
tt0000019| short| 0| 1898| \N| Comedy,Short|
tt0000022| short| 0| 1895| 1| Documentary,Short|
tt0000027| short| 0| 1895| 1| Documentary,Short|
tt0000028| short| 0| 1895| 1| Documentary,Short|
tt0000029| short| 0| 1895| 1| Documentary,Short|
tt0000033| short| 0| 1895| 1|Comedy,Documentar...|
tt0000038| short| 0| 1898| \N|Documentary,Short...|
tt0000039| short| 0| 1896| \N| Short|
tt0000040| short| 0| 1896| \N| Documentary,Short|
tt0000042| short| 0| 1896| \N| Documentary,Short|
tt0000043| short| 0| 1896| \N| Documentary,Short|
tt0000050| short| 0| 1896| \N| Documentary,Short|
tt0000052| short| 0| 1896| 1| Documentary,Short|
tt0000055| short| 0| 1896| \N| Short|
tt0000056| short| 0| 1896| \N| Short|
+---------+---------+-------+---------+--------------+--------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import split, array_distinct, explode
df_basics_sample_fs = df_basics_sample_fs.select('tconst', 'genres', explode(array_distinct(split('genres', ',')))).groupBy('tconst').pivot('col').count().fillna(0).show()


+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
 tconst|Action|Adult|Adventure|Animation|Biography|Comedy|Crime|Documentary|Drama|Family|Fantasy|Film-Noir|Game-Show|History|Horror|Music|Musical|Mystery|News|Reality-TV|Romance|Sci-Fi|Short|Sport|Talk-Show|Thriller|War|Western| \N|
+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
tt0013305| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0089167| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0235483| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0245237| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0331671| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0349113| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0493282| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0544067| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0674313| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0716421| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0028810| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0090978| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
tt0220939| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0|
tt0336404| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0386313| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0421120| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0440947| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
tt0505332| 0| 0| 0| 0| 0| 1| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0566470| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
tt0592174| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0|
+---------+------+-----+---------+---------+---------+------+-----+-----------+-----+------+-------+---------+---------+-------+------+-----+-------+-------+----+----------+-------+------+-----+-----+---------+--------+---+-------+---+
only showing top 20 rows

In [0]:
def basics(dataframe):
  
  droplist = ['primaryTitle', 'originalTitle', 'endYear']
  df_basics_sample_fs = df_basics.drop(*droplist)
  
  df_basics_sample_pivot = df_basics_sample_fs.select('tconst', 'genres', explode(array_distinct(split('genres', ',')))).groupBy('tconst').pivot('col').count().fillna(0)
  df_basics_sample_pivot = df_basics_sample_pivot.drop(r'\N')
                                                                              
  df_ids = dataframe.join(df_basics_sample_pivot, on='tconst')\
     
  return df_ids